<a href="https://colab.research.google.com/github/AsminiKOD/ClientPredicting/blob/RandomForest/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Random Forest Model

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import pandas as pd

# Load the dataset
file_path = 're-duplicates-handled.csv'
data = pd.read_csv(file_path)

# Drop rows with NaN in the target variable
data.dropna(subset=['y'], inplace=True)

# Separate features and target variable
X = data.drop(columns=['y'])
y = data['y']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the Random Forest model with hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced']
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_rf_model = grid_search.best_estimator_

# Evaluate the model on training and testing data
y_train_pred = best_rf_model.predict(X_train_resampled)
y_test_pred = best_rf_model.predict(X_test)

train_report = classification_report(y_train_resampled, y_train_pred)
test_report = classification_report(y_test, y_test_pred)

print("Training Set Classification Report:")
print(train_report)

print("Testing Set Classification Report:")
print(test_report)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Training Set Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     27437
           1       1.00      1.00      1.00     27437

    accuracy                           1.00     54874
   macro avg       1.00      1.00      1.00     54874
weighted avg       1.00      1.00      1.00     54874

Testing Set Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      6860
           1       0.93      0.92      0.92      6691

    accuracy                           0.92     13551
   macro avg       0.92      0.92      0.92     13551
weighted avg       0.92      0.92      0.92     13551

